In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P, Utilities as U
from mppi.Calculators import Tools
from mppi.Datasets import PostProcessing as PP
from mppi.Utilities import Constants as Const
from mppi.Utilities import LatticeUtils as LL
import matplotlib.pyplot as plt
import numpy as np
import os, yaml
from scipy.signal import argrelextrema

In [3]:
# RunRules for ismhpc
# The product of ntasks_per_node*cpus_per_task is equal to 32. 
# Many mpi are needed for better performances
nodes = 1
ntasks_per_node = 16
cpus_per_task=2
omp_num_threads=2
module = '/home/dalessandro/module_script/qe_module'

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='all12h',
                memory='125000',time='11:59:00',
                nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads,pre_processing=module)
code = C.QeCalculator(rr,activate_BeeOND=True) #,skip=False,clean_restart=False
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 16,
 'cpus_per_task': 2,
 'omp_num_threads': 2,
 'gpus_per_node': None,
 'memory': '125000',
 'time': '11:59:00',
 'partition': 'all12h',
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'pre_processing': '/home/dalessandro/module_script/qe_module',
 'executable': 'pw.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': True,
 'verbose': True}

In [5]:
# RunRules for m100
nodes = 1
ntasks_per_node = 4
cpus_per_task=32
omp_num_threads=8
gpus_per_node=4
pe=8

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='m100_usr_prod',
                account='IscrC_AlNWS2',
                memory='120GB',time='23:59:00',
                nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,omp_num_threads=omp_num_threads,
                gpus_per_node=gpus_per_node,pe=pe,
                map_by='socket',rank_by='core')
code = C.QeCalculator(rr)
#code.global_options()

# DFT analysis of the AGSL5

This notebook contains the DFT analysis of the Armchair Graphene SuperLattice with 5 C atoms between each H atom along the Y (armchair) direction performed with QuantumESPRESSO

## GS analysis

We use the input file 'Input_Vasil/qe_scf.in' as starting point for the DFT analysis

In [5]:
inp = I.PwInput(file='Input_Vasil/qe_scf.in')
inp.set_pseudo_dir('../pseudos')
inp

{'control': {'calculation': "'scf'",
  'verbosity': "'high'",
  'prefix': "'pwscf'",
  'outdir': "'./'",
  'restart_mode': "'from_scratch'",
  'wf_collect': '.false.',
  'pseudo_dir': "'../pseudos'"},
 'system': {'force_symmorphic': '.false.',
  'ibrav': 0,
  'nat': 14,
  'ntyp': 2,
  'ecutwfc': 70,
  'nbnd': 35,
  'occupations': "'fixed'",
  'degauss': 0.0,
  'smearing': "'gaussian'"},
 'electrons': {'diago_full_acc': '.false.'},
 'ions': {},
 'cell': {},
 'atomic_species': {'C': [12.011, 'C.upf'], 'H': [1.0, 'H.upf']},
 'atomic_positions': {'type': '(angstrom)',
  'values': [['C', [-1.792510823, 1.4456517994, -0.0414672295]],
   ['C', [0.6379373039, 1.4235971792, -0.3172245562]],
   ['C', [3.0657768079, 1.4455042995, -0.0157591793]],
   ['C', [-0.5959562906, 0.7133098684, -0.2503074405]],
   ['C', [1.8700368141, 0.7135148531, -0.2270262819]],
   ['C', [-3.0402753064, -0.7719313527, 0.4823609916]],
   ['H', [-3.0239104491, -1.0594331893, 1.5579425823]],
   ['C', [-0.5959562909, -0.713